In [ ]:
# ===============================
# 기본 라이브러리
# ===============================
import os
import re
import math
from collections import Counter

# ===============================
# 시각화
# ===============================
import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud

# ===============================
# 데이터 처리
# ===============================
import pandas as pd\

# ===============================
# matplotlib 한글 폰트 설정
# ===============================
font_path = "C:/Windows/Fonts/malgun.ttf"

if not os.path.exists(font_path):
    raise FileNotFoundError("❌ malgun.ttf 폰트를 찾을 수 없습니다.")

font_name = mpl.font_manager.FontProperties(fname=font_path).get_name()
mpl.rcParams["font.family"] = font_name
mpl.rcParams["axes.unicode_minus"] = False




## 데이터 로드

In [ ]:
df = pd.read_csv("./news_data/keyword_merged.csv")

# 필수 컬럼 확인
required_cols = ["검색어", "키워드"]
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"[ERROR] '{col}' 컬럼이 없습니다.")


## 함수 선언

In [ ]:
#############################################
# 2. UI helper 함수
#############################################
def format_search_query(search_word):
    """
    반도체_전망_20231201_20231231
    → 검색어: 반도체 전망 (2023.12.01 ~ 2023.12.31)
    """
    try:
        industry, keyword, start, end = search_word.split("_")
        return f"검색어: {industry} {keyword} ({start[:4]}.{start[4:6]}.{start[6:]} ~ {end[:4]}.{end[4:6]}.{end[6:]})"
    except:
        return f"검색어: {search_word}"


#############################################
# 3. 토큰 정규화 (🔥 모든 비교 기준)
#############################################
def normalize_token(token: str) -> str:
    token = token.lower()
    token = re.sub(r"[^\w가-힣]", "", token)
    return token


#############################################
# 4. 불용어 정의
#############################################
BASE_STOPWORDS = {
    # =========================
    # 1. 일반 기능어 / 조사
    # =========================
    "것","수","등","및","대한","위한","관련","경우","수준","중심",
    "통해","대해","비해","따라","이상","이하","내외","전반",
    "부분","측면","방면","이유","배경","과정","방안","내용",

    # =========================
    # 2. 뉴스·리포트 관용어
    # =========================
    "전망","현황","동향","분석","보고","보고서",
    "산업","기업","기술","시장","구조","환경","영향",
    "확대","강화","추진","대응","활용","기반","가능성",
    "확보","강조","설명","언급","지적","평가","확인",
    "예상","판단","진단","제시","논의","발표","공개",

    # =========================
    # 3. 주체·추상 집단
    # =========================
    "한국","정부","국가","국민","재계","업계",
    "기업들","사람","조직","당국","관계자","전문가",
    "기관","협회","단체","사업","부문",

    # =========================
    # 4. 시점·시간 표현
    # =========================
    "올해","내년","작년","최근","향후","현재",
    "이날","이후","이전","당시","이번","지난해",
    "상반기","하반기","연간","분기","월간","중장기",

    # =========================
    # 5. 기사 메타 / 출처
    # =========================
    "인터뷰","기획","특별","보도","뉴스",
    "기자","취재","사진","제공","연합뉴스","로이터",

    # =========================
    # 6. 너무 포괄적인 평가·수식어
    # =========================
    "주요","핵심","다양한","새로운","높은","낮은",
    "큰","작은","일부","다수","여러",
    "전반적","대표적","일반적","중요","필요",

    # =========================
    # 국가명
    # =========================

    "한국","미국","중국","일본","유럽","eu",

    # 아시아
    "아시아","동남아","베트남","인도","대만",

    # 유럽 주요국
    "독일","프랑스","영국","이탈리아","스페인",

    # 기타 자주 등장
    "러시아","우크라이나","중동","아프리카",

    # 지역 표현
    "글로벌","전세계","해외","국내"


}


# 검색어 의미 중복 제거용
QUERY_SEMANTIC_STOPWORDS = {
    "이차전지": {"이차전지", "2차전지", "이차", "전지"},
    "반도체": {"반도체", "칩", "chip"},
    "디스플레이": {"디스플레이", "패널"},
}

# 산업 연관 기업 (🔥 전역 제거 대상)
RELATED_COMPANIES = {
    "반도체": ["삼성", "삼성전자", "sk", "sk하이닉스", "tsmc", "인텔"],
    "디스플레이": ["lg", "lg디스플레이", "삼성디스플레이"],
    "이차전지": ["lg에너지솔루션", "삼성sdi", "sk온", "catl"],
}



#############################################
# 5. 검색어 파싱 (단일 기준)
#############################################
def parse_search_word(search_word):
    if "_" in search_word:
        parts = search_word.split("_")
        return parts[0], parts[1]

    for industry in QUERY_SEMANTIC_STOPWORDS.keys():
        if industry in search_word:
            return industry, search_word.replace(industry, "")

    return search_word, ""


#############################################
# 6. 패턴 추출
#############################################

def extract_all_query_patterns(search_word):
    """
    검색어 자체 + 의미적으로 같은 단어 전부 제거
    """
    industry, keyword = parse_search_word(search_word)

    patterns = {
        normalize_token(industry),
        normalize_token(keyword)
    }

    patterns |= {
        normalize_token(w)
        for w in QUERY_SEMANTIC_STOPWORDS.get(industry, set())
    }

    return patterns


def extract_all_company_patterns():
    """
    산업 구분 없이 모든 연관 기업 전부 제거
    """
    patterns = set()
    for companies in RELATED_COMPANIES.values():
        for c in companies:
            patterns.add(normalize_token(c))
    return patterns

In [ ]:


#############################################
# 7. 🔥 핵심 키워드 정제 함수
#############################################
def clean_keywords(keyword_series, search_word):
    base_stopwords = {normalize_token(w) for w in BASE_STOPWORDS}
    query_patterns = extract_all_query_patterns(search_word)
    company_patterns = extract_all_company_patterns()

    cleaned = []

    for text in keyword_series:
        if not isinstance(text, str):
            continue

        for w in text.split(","):
            token = normalize_token(w)

            # 1️⃣ 너무 짧은 단어 제거
            if len(token) <= 1:
                continue

            # 2️⃣ 기본 불용어 제거
            if token in base_stopwords:
                continue

            # 3️⃣ 검색어 + 의미 중복 제거
            if any(p in token for p in query_patterns):
                continue

            # 4️⃣ 검색어 연관 기업 제거 (🔥 핵심)
            if any(c in token for c in company_patterns):
                continue

            cleaned.append(token)

    return cleaned



#############################################
# 8. 워드클라우드 실행
#############################################

for search_word, group in df.groupby("검색어"):

    group = group.dropna(subset=["키워드"])

    words = clean_keywords(group["키워드"], search_word)
    if not words:
        continue

    # -----------------------------
    # 키워드 빈도
    # -----------------------------
    freq = Counter(words)
    top_words = dict(freq.most_common(20))

    # -----------------------------
    # 워드클라우드
    # -----------------------------
    wc = WordCloud(
        font_path=font_path,
        background_color="white",
        width=1000,
        height=520,
        max_words=20,
        relative_scaling=0.25,
        colormap="viridis"  # 🔥 색감 통일
    ).generate_from_frequencies(top_words)

    # -----------------------------
    # 시각화
    # -----------------------------
    fig = plt.figure(figsize=(15, 8), facecolor="white")

    # 메인 타이틀
    fig.text(
        0.05, 0.94,
        "산업전망 현황",
        fontsize=18,
        fontweight="bold",
        ha="left",
        color="#222222"
    )

    # 서브 타이틀 (검색어)
    fig.text(
        0.05, 0.90,
        format_search_query(search_word),
        fontsize=12,
        ha="left",
        color="#555555"
    )

    # 구분선
    fig.lines.append(
        plt.Line2D(
            [0.05, 0.95], [0.875, 0.875],
            linewidth=1,
            color="#DDDDDD"
        )
    )

    # 워드클라우드 영역
    ax = fig.add_axes([0.05, 0.10, 0.90, 0.72])
    ax.imshow(wc)
    ax.axis("off")

    plt.show()


### 감성분석 포함 
- WC 에는 명사만, 감성분석할 때에는 동사로만

In [ ]:
#############################################
# 9. 동사 기반 감성 사전
#############################################

POS_VERBS = {
    "성장", "확대", "증가", "회복", "개선",
    "강화", "확보", "상승", "확장", "활성화",
    "호조", "진전", "개편", "도약", "안정",
    "성공", "확산", "정착", "선도", "촉진"
}


NEG_VERBS = {
    "감소", "부진", "위축", "둔화", "하락",
    "악화", "축소", "정체", "리스크", "불확실",
    "차질", "지연", "감퇴", "침체", "부담",
    "불안", "악재", "제약", "후퇴", "손실"
}


In [ ]:
#############################################
# 10. 키워드 문자열에서 동사만 추출
#############################################

def extract_verbs(keyword_series):
    verbs = []

    pos_verbs = {normalize_token(v) for v in POS_VERBS}
    neg_verbs = {normalize_token(v) for v in NEG_VERBS}

    for text in keyword_series:
        if not isinstance(text, str):
            continue

        for w in text.split(","):
            token = normalize_token(w)

            if token in pos_verbs or token in neg_verbs:
                verbs.append(token)

    return verbs


In [ ]:
#############################################
# 11. 동사 기반 감성 점수 계산
#############################################

def calculate_sentiment_from_verbs(verbs):
    if not verbs:
        return 0.0

    score = 0
    for v in verbs:
        if v in POS_VERBS:
            score += 1
        elif v in NEG_VERBS:
            score -= 1

    # 평균 감성 점수
    return round(score / len(verbs), 3)


- 평균 감성 점수
    - [정의] (긍정 동사 수 − 부정 동사 수) ÷ 전체 동사 수

- 범위: -1 ~ +1
---

In [ ]:
#############################################
# 13. 긍정 / 부정 동사 Top 리스트
#############################################

def get_top_verbs(verbs, top_n=5):
    pos = Counter(v for v in verbs if v in POS_VERBS)
    neg = Counter(v for v in verbs if v in NEG_VERBS)

    return (
        pos.most_common(top_n),
        neg.most_common(top_n)
    )


In [ ]:
from collections import Counter

def get_top_positive_verbs(verbs, top_n=5):
    pos_verbs = {normalize_token(v) for v in POS_VERBS}
    pos_only = [v for v in verbs if v in pos_verbs]
    return Counter(pos_only).most_common(top_n)

def get_top_negative_verbs(verbs, top_n=5):
    neg_verbs = {normalize_token(v) for v in NEG_VERBS}
    neg_only = [v for v in verbs if v in neg_verbs]
    return Counter(neg_only).most_common(top_n)


In [ ]:
def extract_verbs(keyword_series):
    verbs = []

    pos_verbs = {normalize_token(v) for v in POS_VERBS}
    neg_verbs = {normalize_token(v) for v in NEG_VERBS}

    for text in keyword_series:
        if not isinstance(text, str):
            continue

        for w in text.split(","):
            token = normalize_token(w)

            if token in pos_verbs or token in neg_verbs:
                verbs.append(token)

    return verbs


In [ ]:
def sentiment_summary(verbs):
    pos = sum(1 for v in verbs if v in POS_VERBS)
    neg = sum(1 for v in verbs if v in NEG_VERBS)
    total = pos + neg

    if total == 0:
        return 0.0, 0.0, pos, neg

    avg_score = (pos - neg) / total
    pos_ratio = pos / total

    return round(avg_score, 2), round(pos_ratio, 2), pos, neg


In [ ]:
from collections import Counter

def sentiment_confidence(verbs):
    """
    감성 점수 신뢰도 판단
    기준:
    1) 동사 수
    2) 긍/부정 비율 편향
    3) 상위 동사 쏠림
    """

    n = len(verbs)

    # ① 표본 수 부족
    if n < 5:
        return "낮음"

    # 긍/부정 개수
    pos_set = {normalize_token(v) for v in POS_VERBS}
    neg_set = {normalize_token(v) for v in NEG_VERBS}

    pos_cnt = sum(1 for v in verbs if v in pos_set)
    neg_cnt = sum(1 for v in verbs if v in neg_set)

    total = pos_cnt + neg_cnt
    if total == 0:
        return "낮음"

    pos_ratio = pos_cnt / total

    # ② 방향성 불명확 (너무 섞여 있음)
    if 0.4 <= pos_ratio <= 0.6:
        base_conf = "보통"
    else:
        base_conf = "높음"

    # ③ 특정 동사 쏠림 체크
    freq = Counter(verbs)
    top_ratio = freq.most_common(1)[0][1] / n

    if top_ratio > 0.6:
        # 한 단어에 과도하게 의존
        return "보통" if base_conf == "높음" else "낮음"

    return base_conf


In [ ]:
#############################################
# 7. 🔥 핵심 키워드 정제 함수 (명사용)
#############################################
def clean_keywords(keyword_series, search_word):
    base_stopwords = {normalize_token(w) for w in BASE_STOPWORDS}
    query_patterns = extract_all_query_patterns(search_word)
    company_patterns = extract_all_company_patterns()

    # 🔥 동사 사전 (워드클라우드에서는 제거)
    verb_patterns = {normalize_token(v) for v in POS_VERBS | NEG_VERBS}

    cleaned = []

    for text in keyword_series:
        if not isinstance(text, str):
            continue

        for w in text.split(","):
            token = normalize_token(w)

            # 1️⃣ 너무 짧은 단어 제거
            if len(token) <= 1:
                continue

            # 2️⃣ 기본 불용어
            if token in base_stopwords:
                continue

            # 3️⃣ 검색어/의미 중복 제거
            if any(p in token for p in query_patterns):
                continue

            # 4️⃣ 기업명 제거
            if any(c in token for c in company_patterns):
                continue

            # 🔥 5️⃣ 동사 제거 (핵심)
            if token in verb_patterns:
                continue

            cleaned.append(token)

    return cleaned


In [ ]:
#############################################
# 12. 워드클라우드 + 평균 감성 점수 UI
#############################################

for search_word, group in df.groupby("검색어"):

    group = group.dropna(subset=["키워드"])

    # -----------------------------
    # (1) 명사 → 워드클라우드
    # -----------------------------
    words = clean_keywords(group["키워드"], search_word)
    if not words:
        continue

    freq = Counter(words)
    top_words = dict(freq.most_common(20))

    # wc = WordCloud(
    #     font_path=font_path,
    #     background_color="white",
    #     width=1000,
    #     height=520,
    #     max_words=20,
    #     relative_scaling=0.25,
    #     colormap="cividis"
    # ).generate_from_frequencies(top_words)

    wc = WordCloud(
        font_path=font_path,
        background_color="white",
        width=1000,
        height=520,
        max_words=20,

        # 🔥 spacing & 가독성 핵심
        margin=50,
        relative_scaling=0.25,
        prefer_horizontal=0.95,
        max_font_size=150,

        # 색감
        colormap="magma"
    ).generate_from_frequencies(top_words)



    # -----------------------------
    # (2) 동사 → 감성 점수
    # -----------------------------
    verbs = extract_verbs(group["키워드"])

    avg_sentiment, pos_ratio, pos_cnt, neg_cnt = sentiment_summary(verbs)
    confidence = sentiment_confidence(verbs)

    pos_top = get_top_positive_verbs(verbs, top_n=5)
    neg_top = get_top_negative_verbs(verbs, top_n=5)

    # -----------------------------
    # (3) 시각화
    # -----------------------------
    fig = plt.figure(figsize=(16, 8), facecolor="white")

    # 타이틀
    fig.text(
        0.05, 0.94,
        "산업전망 현황",
        fontsize=18,
        fontweight="bold",
        ha="left"
    )

    # 검색어
    fig.text(
        0.05, 0.90,
        format_search_query(search_word),
        fontsize=12,
        ha="left",
        color="#555555"
    )

    # -----------------------------
    # 워드클라우드 (좌측)
    # -----------------------------
    ax_wc = fig.add_axes([0.05, 0.10, 0.68, 0.75])
    ax_wc.imshow(wc)
    ax_wc.axis("off")

    # -----------------------------
    # 감성 카드 (우측)
    # -----------------------------
    ax_card = fig.add_axes([0.75, 0.18, 0.22, 0.62])
    ax_card.set_xlim(0, 1)
    ax_card.set_ylim(0, 1)
    ax_card.axis("off")

    sentiment_color = (
        "#00008B" if avg_sentiment > 0
        else "#e74c3c" if avg_sentiment < 0
        else "#95a5a6"
    )

    # =============================
    # (1) 평균 감성 점수
    # =============================
    ax_card.text(
        0.5, 0.92,
        "평균 감성 점수",
        fontsize=11,
        ha="center",
        color="#333333"
    )

    ax_card.text(
        0.5, 0.82,
        f"{avg_sentiment:+.2f}",
        fontsize=38,
        fontweight="bold",
        ha="center",
        color=sentiment_color
    )

    ax_card.text(
        0.5, 0.72,
        f"신뢰도: {confidence}",
        fontsize=10,
        ha="center",
        color="#777777"
    )

    # 구분선
    ax_card.plot([0.05, 0.95], [0.66, 0.66], color="#DDDDDD", lw=1)

    # =============================
    # (2) 긍정 / 부정 동사 2컬럼
    # =============================
    # 🟢 긍정
    ax_card.text(0.25, 0.60, "[+] TOP 5",
                fontsize=11, ha="center",
                fontweight="bold", color="#27ae60")

    y_pos = 0.54
    for verb, cnt in pos_top:
        ax_card.text(
            0.25, y_pos,
            f"{verb} ({cnt})",
            fontsize=10,
            ha="center",
            color="#333333"
        )
        y_pos -= 0.06


    # 🔴 부정
    ax_card.text(0.75, 0.60, "[-] TOP 5",
                fontsize=11, ha="center",
                fontweight="bold", color="#c0392b")

    y_neg = 0.54
    for verb, cnt in neg_top:
        ax_card.text(
            0.75, y_neg,
            f"{verb} ({cnt})",
            fontsize=10,
            ha="center",
            color="#333333"
        )
        y_neg -= 0.06

    plt.show()
